In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt

c:\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Device configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [4]:
# 1. Data Loading and Preprocessing
# Define a transform to convert images to tensors.
transform = transforms.Compose([
    transforms.ToTensor(),
    # Optionally, add normalization:
    transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
])

In [5]:
# Download and load the CIFAR-10 training and test datasets.
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)


100%|██████████| 170498071/170498071 [00:41<00:00, 4064704.68it/s]


Extracting ./data\cifar-10-python.tar.gz to ./data
Files already downloaded and verified


In [6]:
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32,
                                          shuffle=True, num_workers=2)
testloader = torch.utils.data.DataLoader(testset, batch_size=32,
                                         shuffle=False, num_workers=2)

In [9]:
# 2. Define the CNN Model
# Here we subclass nn.Module to define a simple CNN with two convolutional layers,
# pooling, and two fully-connected layers.
class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()
        # Convolutional layer 1: from 3 input channels to 32 output channels
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, padding=1)
        # Activation (ReLU) and pooling follow
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        # Convolutional layer 2: from 32 channels to 64 channels
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        # A fully connected layer that takes the flattened features.
        # For CIFAR-10 images of size 32x32, after two poolings (each halving dimensions),
        # the spatial size becomes 8x8, and with 64 channels, that gives 64*8*8 = 4096 features.
        self.fc1 = nn.Linear(64 * 8 * 8, 512)
        self.fc2 = nn.Linear(512, 10)  # 10 output classes

    def forward(self, x):
        # First conv layer + activation + pooling.
        x = self.pool(self.relu(self.conv1(x)))
        # Second conv layer + activation + pooling.
        x = self.pool(self.relu(self.conv2(x)))
        # Flatten the feature map to a vector
        x = x.view(x.size(0), -1)
        # Fully connected layers with activation
        x = self.relu(self.fc1(x))
        # Final fully connected layer (logits output)
        x = self.fc2(x)
        return x


model = SimpleCNN().to(device)
print(model)

SimpleCNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu): ReLU()
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=4096, out_features=512, bias=True)
  (fc2): Linear(in_features=512, out_features=10, bias=True)
)


In [11]:
import torch.optim as optim

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [ ]:
num_epochs = 5

for epoch in range(num_epochs):
    running_loss = 0.0
    for inputs, labels in trainloader:
        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}, Loss: {running_loss/len(trainloader)}")

print("Training complete!")


Epoch 1, Loss: 1.2169259318310865
Epoch 2, Loss: 0.8303578141707301
Epoch 3, Loss: 0.6243801666560725
Epoch 4, Loss: 0.4324473485312474
Epoch 5, Loss: 0.2710882302359347
Training complete!


In [13]:
correct = 0
total = 0

with torch.no_grad():
    for inputs, labels in testloader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        _, predicted = torch.max(outputs, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Accuracy on test set: {accuracy:.2f}%")


Accuracy on test set: 72.57%
